### Import Required Libraries and Set Up Environment Variables

In [75]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [76]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [77]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
partial_url = f"{url}fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}&api-key={nyt_api_key}"


In [78]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(1):  
    # create query with a page number
    # API results show 10 articles at a time
    query_url = f"{partial_url}&page={page}"
    print(query_url)
    
    # Make a "GET" request and retrieve the JSON
    response = requests.get(query_url)
    reviews = response.json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)

    #reviews_list.extend(reviews["response"]["docs"])
    
    try:
        # Try and save the reviews to the reviews_list
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)
            
    except Exception as e:
        # Generic exception handling to capture all issues, such as connection errors
        print(f"Failed to process page {page}, error: {str(e)}")
        break

        # Print the page that was just retrieved
    print(f"Page {page} fetched")
    # Print the page number that had no results then break from the loop


https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count&begin_date=20130101&end_date=20230531&api-key=ukH6PoGwBt9vj3AFAzcm8NcJy0zdVLlN&page=0
Page 0 fetched


In [79]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [80]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
df_nyt = pd.json_normalize(reviews_list)


In [81]:
# Extract the title from the "headline.main" column and

# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
df_nyt['title'] = df_nyt['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])

In [82]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
df_nyt['keywords'] = df_nyt['keywords'].apply(extract_keywords)

In [83]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles_list = df_nyt['title'].tolist()

### Access The Movie Database API

In [84]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [85]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for title in titles_list:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        time.sleep(1)
        print("Napping")

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    query_url = f"{url}{title}{tmdb_key_string}"
    response = requests.get(query_url)
    reviews = response.json()

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    for review in reviews["results"]:
        # Get movie id
        id = review["id"]

        # Make a request for a the full movie details
        details_url = f"https://api.themoviedb.org/3/movie/{id}?api_key={tmdb_api_key}"

        # Execute "GET" request with url
        response = requests.get(details_url)
        details = response.json()
        df = pd.json_normalize(details)

        # Extract the genre names into a list
        genre_names = [genre['name'] for genre in details['genres']]

        # Extract the spoken_languages' English name into a list
        spoken_languages = [language['english_name'] for language in details['spoken_languages']]

        # Extract the production_countries' name into a list
        production_countries = [country['name'] for country in details['production_countries']]


        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        new_details = {}
        new_details["title"] = details["title"]
        new_details["original_title"] = details["original_title"]
        new_details["budget"] = details["budget"]
        new_details["original_language"] = details["original_language"]
        new_details["homepage"] = details["homepage"]
        new_details["overview"] = details["overview"]
        new_details["popularity"] = details["popularity"]
        new_details["runtime"] = details["runtime"]
        new_details["revenue"] = details["revenue"]
        new_details["release_date"] = details["release_date"]
        new_details["vote_average"] = details["vote_average"]
        new_details["vote_count"] = details["vote_count"]

        new_details["genres"] = genre_names
        new_details["spoken_languages"] = spoken_languages
        new_details["production_coutries"] = production_countries

        
        # Print out the title that was found
        print(new_details["title"])

        tmdb_movies_list.append(new_details)


The Attachment Diaries
You Can Live Forever
A Tourist's Guide to Love
Other People's Children
Other People's Children
Other People's Children
Other People's Children
Other People's Children
One True Loves
One True Love
The Lost Weekend: A Love Story
A Thousand and One
A Thousand and One Nights
A Thousand and One Nights
A Thousand and One Nights
A Thousand and One Nights with Toho
A Thousand and One Wives
a thousand and one voices
A Thousand and One Nights
A Thousand and One Monica
A Thousand and One Hands
A Thousand and One Hands
Isabel Allende: A Thousand and One Voices
Anteojito and Antifaz, A Thousand Attempts and One Invention
Alfredo Arias, the man of a thousand and one heads
Miguel Otero Silva: A life and one thousand stories to tell
One in a Thousand
One in a Hundred Thousand
One and Thousand Nights
One Chance in One Thousand
One in a Thousand
Your Place or Mine
Your Place or Mine!
Your Place or Mine?
Your Place... or Mine?
Love in the Time of Fentanyl


In [86]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))

[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "original_language": "es",
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 2.959,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 3.0,
        "vote_count": 4,
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_coutries": [
            "Argentina"
        ]
  

In [91]:
# Convert the results to a DataFrame
df_tmbd = pd.json_normalize(tmdb_movies_list)
df_tmbd.head(5)

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_coutries
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",2.959,102,0,2021-10-07,3.000,4,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",43.566,96,15055,2023-03-24,6.700,33,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
2,A Tourist's Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",19.581,96,0,2023-04-21,6.300,158,"[Romance, Comedy]","[English, Vietnamese]",[United States of America]
3,Other People's Children,Les Enfants des autres,0,fr,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",10.024,104,84178,2022-09-21,6.842,184,"[Drama, Comedy]","[English, French]",[France]
4,Other People's Children,Andre folks børn,0,da,,"The sympathetic farmers couple, Mads and Katri...",0.741,85,0,1958-01-01,4.000,2,[Family],[Danish],[Denmark]


### Merge and Clean the Data for Export

In [94]:
# Merge the New York Times reviews and TMDB DataFrames on title
df_merged = pd.merge(df_tmbd, df_nyt, on='title', how='inner')
df_merged.head(5)

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",2.959,102,0,2021-10-07,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",43.566,96,15055,2023-03-24,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,15.929,100,37820,2023-04-07,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,en,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,2.879,95,77145,2023-04-13,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,A Thousand and One,A Thousand and One,0,en,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,15.202,116,3395595,2023-03-31,...,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [101]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'keywords']

# Create a list of characters to remove
chars_to_remove = ['[', ']', "'", '"']

# Loop through the list of columns to fix
for column in columns_to_fix:

    # Convert the column to type 'str'
    df_merged[column] = df_merged[column].astype(str)

    # Loop through characters to remove
    for char in chars_to_remove:
        df_merged[column] = df_merged[column].str.replace(char, '', regex=False)

# Display the fixed DataFrame
df_merged.head(5)


,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",2.959,102,0,2021-10-07,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",43.566,96,15055,2023-03-24,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,15.929,100,37820,2023-04-07,...,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,en,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,2.879,95,77145,2023-04-13,...,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None
4,A Thousand and One,A Thousand and One,0,en,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,15.202,116,3395595,2023-03-31,...,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,None


In [103]:
# Drop "byline.person" column
#df_merged = df_merged.drop(columns=['byline.person'])

In [105]:
# Delete duplicate rows and reset index
#df_merged = df_merged.drop_duplicates()
#df_merged = df_merged.reset_index(drop=True)


In [107]:
# Export data to CSV without the index
df_merged.to_csv('exported_dataframe.csv', index=False)